In [104]:
from __future__ import division
import pandas as pd
from numpy import random
# import dask.dataframe as dd

# Loading train df

Загрузка `train_df` без признака `row_id` - дублирует индекс

In [105]:
chunksize = 10 ** 5
dtypes = {
            'timestamp': 'int64',
            'user_id': 'int32',
            'content_id': 'int16',
            'content_type_id': 'int8',
            'task_container_id': 'int16',
            'user_answer': 'int8',
            'answered_correctly':'int8',
            'prior_question_elapsed_time': 'float32',
            'prior_question_had_explanation': 'object'
            }
train_path = '/home/ksu/Desktop/magistr/homework/HW_2/data/train.csv'
if chunksize:
    chunks = pd.read_csv(train_path, usecols=[1,2,3,4,5,6,7,8,9],
                         dtype=dtypes, chunksize=chunksize, low_memory=False)
    train_df = pd.DataFrame(chunks.get_chunk(chunksize))
    # df=pd.concat(chunk for chunk in chunks)
else:
    train_df = pd.read_csv(train_path, usecols=[1,2,3,4,5,6,7,8,9], dtype=dtypes)

## Data cleaning
- убираются пустые ячейки (строка 1)
- признак `prior_question_had_explanation` преобразуется в `int8` (строки 2 - 6)
- признак `prior_question_elapsed_time` преобразуется в секунды. Так как после этого данные укладываются в 2 байта
признак преобразуется в `int16`

In [106]:
train_df.dropna(inplace=True)
train_df['prior_question_had_explanation'] = (
    train_df['prior_question_had_explanation'].
    apply(lambda has_explanation: 1 if has_explanation == 'True' else 0)
)
train_df['prior_question_had_explanation'] = train_df['prior_question_had_explanation'].astype('int8')
# now prior_question_elapsed_time will be in seconds
train_df['prior_question_elapsed_time'] = train_df['prior_question_elapsed_time'] / 1e3
print('prior_question_elapsed_time min value: {} max value {}\n'.format(
                train_df['prior_question_elapsed_time'].min(),
                train_df['prior_question_elapsed_time'].max()))
train_df['prior_question_elapsed_time'] = train_df['prior_question_elapsed_time'].astype('int16')
train_df.info(memory_usage=200, null_counts=True)

prior_question_elapsed_time min value: 0.0 max value 300.0

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97834 entries, 1 to 99999
Data columns (total 9 columns):
timestamp                         97834 non-null int64
user_id                           97834 non-null int32
content_id                        97834 non-null int16
content_type_id                   97834 non-null int8
task_container_id                 97834 non-null int16
user_answer                       97834 non-null int8
answered_correctly                97834 non-null int8
prior_question_elapsed_time       97834 non-null int16
prior_question_had_explanation    97834 non-null int8
dtypes: int16(3), int32(1), int64(1), int8(4)
memory usage: 2.8 MB


# Loading questions and lectures data

In [107]:
questions_df = pd.read_csv('/home/ksu/Desktop/magistr/homework/HW_2/data/questions.csv')
lectures_df = pd.read_csv('/home/ksu/Desktop/magistr/homework/HW_2/data/lectures.csv')

## Merging data

In [108]:
train = pd.merge(train_df, questions_df, left_on = 'content_id', right_on = 'question_id', how = 'left')
train = pd.merge(train, lectures_df, left_on = 'content_id', right_on = 'lecture_id', how = 'left')
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97834 entries, 0 to 97833
Data columns (total 18 columns):
timestamp                         97834 non-null int64
user_id                           97834 non-null int32
content_id                        97834 non-null int16
content_type_id                   97834 non-null int8
task_container_id                 97834 non-null int16
user_answer                       97834 non-null int8
answered_correctly                97834 non-null int8
prior_question_elapsed_time       97834 non-null int16
prior_question_had_explanation    97834 non-null int8
question_id                       97834 non-null int64
bundle_id                         97834 non-null int64
correct_answer                    97834 non-null int64
part_x                            97834 non-null int64
tags                              97834 non-null object
lecture_id                        1244 non-null float64
tag                               1244 non-null float64
part_y      

In [109]:
def calc_percents(dlen, flen):
    return 100 * flen / dlen

dlen = train_df.shape[0]

## timestamp

the time in milliseconds between this user interaction and the first event completion from that user

In [110]:
timestamp_secs = train_df['timestamp']/1e3
print('Timestamp max {:.1f}days min {:.1f}s mode {:.1f}s'.format(timestamp_secs.max()/(3600 * 24),
                                                                    timestamp_secs.min(),
                                                                    timestamp_secs.mode()[0]))
zero_ts_len = len(timestamp_secs[timestamp_secs == 0])
hour_ts_len = len(timestamp_secs[timestamp_secs > 3600])
day_ts_len = len(timestamp_secs[timestamp_secs > 3600 * 24])
print('Zero timestamp (or interaction = event, or garbage) amount {}, in percents: {:.2f}%'.format(zero_ts_len, calc_percents(dlen, zero_ts_len)))
print('Timestamps which are more than 1 hour amount {}, in percents: {:.1f}%'.format(hour_ts_len, calc_percents(dlen, hour_ts_len)))
print('Timestamps which are more than 1 day amount {}, in percents: {:.1f}%'.format(day_ts_len, calc_percents(dlen, day_ts_len)))

Timestamp max 688.6days min 0.0s mode 341.4s
Zero timestamp (or interaction = event, or garbage) amount 4, in percents: 0.00%
Timestamps which are more than 1 hour amount 89816, in percents: 91.8%
Timestamps which are more than 1 day amount 85640, in percents: 87.5%


## user_id

ID code for the user

In [111]:
user_id_unique = len(train_df['user_id'].unique())
print('Amount of unquie users {}, percents of df length {:.1f}%'.format(user_id_unique, calc_percents(dlen, user_id_unique)))

Amount of unquie users 348, percents of df length 0.4%


## content_id

ID code for the user interaction

In [112]:
content_id_len = len(train_df['content_id'].unique())
print('Amount of unquie content ids {}, percents of df length {:.2f}%'.format(content_id_len, calc_percents(dlen, content_id_len)))

Amount of unquie content ids 11308, percents of df length 11.56%


## task_container_id

Id code for the batch of questions or lectures. For example, a user might see three questions in a row before seeing the explanations for any of them. Those three would all share a task_container_id.

Questions topics (themes)?

In [113]:
task_container_id_len = len(train_df['task_container_id'].unique())
print('Amount of unquie task containers ids {}, percents of df length {:.2f}%'.format(
    task_container_id_len,
    calc_percents(dlen, task_container_id_len)))

Amount of unquie task containers ids 5631, percents of df length 5.76%


## content_type_id

0 if the event was a question being posed to the user, 1 if the event was the user watching a lecture

In [114]:
train_df['content_type_id'].value_counts(normalize=True)

0    1.0
Name: content_type_id, dtype: float64

all content is questions, so this data is not usefull

In [115]:
train_df.drop(['content_type_id'], axis=1, inplace=True)
train_df.head()

,timestamp,user_id,content_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
1,56943,115,5716,2,2,1,37,0
2,118363,115,128,0,0,1,55,0
3,131167,115,7860,3,0,1,19,0
4,137965,115,7922,4,1,1,11,0
5,157063,115,156,5,2,1,5,0


## user_answer

the user's answer to the question, if any. Read -1 as null, for lectures.

In [116]:
train_df['user_answer'].value_counts()

0    27759
1    26428
3    25984
2    17663
Name: user_answer, dtype: int64

## answered_correctly

if the user responded correctly. Read -1 as null, for lectures

In [117]:
train_df['answered_correctly'].value_counts()

1    67100
0    30734
Name: answered_correctly, dtype: int64

## prior_question_had_explanation

Whether or not the user saw an explanation and the correct response(s) after answering the previous question bundle, ignoring any lectures in between. The value is shared across a single question bundle, and is null for a user's first question bundle or lecture. Typically the first several questions a user sees were part of an onboarding diagnostic test where they did not get any feedback

In [118]:
train_df['prior_question_had_explanation'].value_counts()

1    90349
0     7485
Name: prior_question_had_explanation, dtype: int64

## prior_question_elapsed_time

 The average time in milliseconds it took a user to answer each question in the previous question bundle, ignoring any lectures in between. Is null for a user's first question bundle or lecture. Note that the time is the average time a user took to solve each question in the previous bundle

In [119]:
print('Elapsed time min: {} max: {} mode: {}'.format(train_df['prior_question_elapsed_time'].min(),
                                                  train_df['prior_question_elapsed_time'].max(),
                                                  train_df['prior_question_elapsed_time'].mode()[0]))
train_df['prior_question_elapsed_time'].value_counts()

Elapsed time min: 0 max: 300 mode: 17


17     5809
16     5449
18     5168
19     4507
20     4204
15     4126
22     4069
21     4059
23     3773
24     3599
14     3073
25     3058
26     2823
27     2483
13     2322
12     2184
28     2047
29     1986
11     1856
31     1631
10     1617
30     1613
9      1580
32     1415
8      1383
33     1221
34     1197
7      1062
35     1061
6       898
       ... 
170       1
153       1
189       1
281       1
190       1
184       1
151       1
175       1
201       1
176       1
241       1
203       1
275       1
165       1
266       1
212       1
228       1
265       1
244       1
147       1
269       1
272       1
213       1
226       1
209       1
135       1
239       1
224       1
262       1
146       1
Name: prior_question_elapsed_time, Length: 220, dtype: int64

## question_id

foreign key for the train/test content_id column, when the content type is question (0).

In [120]:
questions_df['question_id'].head()

0    0
1    1
2    2
3    3
4    4
Name: question_id, dtype: int64

In [121]:
questions_df.drop(['question_id'], inplace=True, axis=1)
questions_df.head()

,bundle_id,correct_answer,part,tags
0,0,0,1,51 131 162 38
1,1,1,1,131 36 81
2,2,0,1,131 101 162 92
3,3,0,1,131 149 162 29
4,4,3,1,131 5 162 38


## bundle_id

code for which questions are served together

In [122]:
questions_df['bundle_id'].value_counts().head()

7795    5
6971    5
7421    5
7770    5
8144    5
Name: bundle_id, dtype: int64

## correct_answer

the answer to the question. Can be compared with the train user_answer column to check if the user was right.

In [123]:
questions_df['correct_answer'].value_counts()

0    3716
3    3544
1    3478
2    2785
Name: correct_answer, dtype: int64

## part

the relevant section of the TOEIC test.

https://www.iibc-global.org/english/toeic/test/lr/about/format.html

In [124]:
questions_df['part'].value_counts()

5    5511
2    1647
3    1562
4    1439
6    1212
7    1160
1     992
Name: part, dtype: int64

## tag

one or more detailed tag codes for the question. The meaning of the tags will not be provided, but these codes are sufficient for clustering the questions together.

In [125]:
questions_df['tags'].head()

0     51 131 162 38
1         131 36 81
2    131 101 162 92
3    131 149 162 29
4      131 5 162 38
Name: tags, dtype: object

## type_of

brief description of the core purpose of the lecture

In [126]:
lectures_df['type_of'].value_counts()

concept             222
solving question    186
intention             7
starter               3
Name: type_of, dtype: int64

## part

top level category code for the lecture.

https://www.iibc-global.org/english/toeic/test/lr/about/format.html

In [127]:
lectures_df['part'].value_counts()

5    143
6     83
2     56
1     54
7     32
4     31
3     19
Name: part, dtype: int64

## tag

one tag codes for the lecture. The meaning of the tags will not be provided, but these codes are sufficient for clustering the lectures together

In [132]:
train['tag']

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
5         NaN
6         NaN
7         NaN
8         NaN
9         NaN
10        NaN
11        NaN
12        NaN
13        NaN
14        NaN
15        NaN
16        NaN
17       57.0
18        NaN
19        NaN
20        NaN
21        NaN
22       45.0
23        NaN
24        NaN
25        NaN
26        NaN
27        NaN
28        NaN
29       70.0
         ... 
97804     NaN
97805     NaN
97806     NaN
97807     NaN
97808     NaN
97809     NaN
97810     NaN
97811     NaN
97812     NaN
97813     NaN
97814     NaN
97815     NaN
97816     NaN
97817     NaN
97818     NaN
97819     NaN
97820     NaN
97821     NaN
97822     NaN
97823     NaN
97824     NaN
97825     NaN
97826     NaN
97827     NaN
97828     NaN
97829     NaN
97830     NaN
97831     NaN
97832     NaN
97833     NaN
Name: tag, Length: 97834, dtype: float64